In [1]:
def load_txt(filename):
    with open(filename,'r') as df:
        lines = df.readlines()
        data = []
        for line in lines:
            tem = line.split()
            x = [float(i) for i in tem]
            data.append(x)
    return data

In [2]:
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right


In [3]:
def gini_value(groups, classes):
    n = float(sum([len(group) for group in groups]))
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p**2
        gini += (1.0 - score) * (size / n)
    return gini

In [4]:
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_value(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [5]:
def exit_all(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

In [6]:
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    if not left or not right:
        node['left'] = node['right'] = exit_all(left + right)
        return
    if depth >= max_depth:
        node['left'], node['right'] = exit_all(left), exit_all(right)
        return
    if len(left) <= min_size:
        node['left'] = exit_all(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    if len(right) <= min_size:
        node['right'] = exit_all(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

In [7]:
def buildDt(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

In [8]:
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [9]:
def decision_tree(train, test, max_depth, min_size):
    tree = buildDt(train, max_depth, min_size)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return(predictions)

In [10]:
def Eout(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] != predicted[i]:
            correct += 1
    return correct / float(len(actual))

In [21]:
def evaluate(train_set, test_set,algorithm,*arg):
    
    predicted = algorithm(train_set, test_set, *arg)
    actual = [row[-1] for row in test_set]
    accuracy = Eout(actual, predicted)
    
    return accuracy

In [17]:
dataset = load_txt('train.txt')
test_set = load_txt('test.txt')
max_depth = 20
min_size = 1
scores = evaluate(dataset,test_set, decision_tree, max_depth, min_size)
print('Q14: %s' % scores)

Q14: 0.174
